# 한국 부동산 시장 분석 및 투자 전략 수립

## 자유주제: 금리 및 정부 부동산 정책 변화가 서울 부동산 시장에 미치는 영향 분석
1. 금리 데이터 크롤링
    - 금리 데이터와 부동산 시장(거래량, 거래금액)과의 상관관계 분석
2. 주요 정부 부동산 정책 날짜 체크
    - 날짜 기준으로 잘라서 앞뒤 따로 분석(그래프 등)
3. 실무 인사이트 도출

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

df_sales = pd.read_csv('./data/sales_clean.csv')
df_long = pd.read_csv('./data/long_clean.csv')
df_monthly = pd.read_csv('./data/monthly_clean.csv')